In [3]:
import requests
import json
import pandas as pd
import time

# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

# Running flask from terminal on OSX
# export FLASK_APP=api
# export FLASK_ENV=development
# flask run

# Running with gunicorn (heroku, digitalocean, etc):
# Procfile >
# web: gunicorn wsgi --log-file -
# wsgi.py >
# from api import app as application
#
#if __name__ == "__main__":
#    application.run()

# https://devcenter.heroku.com/articles/getting-started-with-python
# https://www.digitalocean.com/community/tutorials/how-to-serve-flask-applications-with-gunicorn-and-nginx-on-ubuntu-14-04

In [4]:
# Set up heroku database > 
# heroku addons:create heroku-postgresql:hobby-dev
# Postgres db now up
# Look at config vars (should have DATABASE_URL) >
# heroku config
# Connect to remote psql > 
# heroku pg:psql
#


In [146]:
#  baseurl = 'https://vast-depths-89393.herokuapp.com/'
baseurl = 'https://glacial-hollows-90395.herokuapp.com/'
#baseurl = 'http://127.0.0.1:5000/'

In [147]:
url = baseurl +'createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [148]:
response.json()

{'signingKey_hex': '0935140d7a16472d9af3252ce8eb37567eacfc49fa43f699456beb7e6f7174dc',
 'traderId': '7',
 'verifyKey_hex': '88cffa0814aa272e93df7712986ddb9cede6f63fef0673b85b0c9bd203dab202'}

In [149]:
url = baseurl + 'createMarket'

mkId = 6
# copy from response
sk = response.json()['signingKey_hex']
vk = response.json()['verifyKey_hex']
tId = response.json()['traderId']

In [150]:
content_makemarket = {"signingKey_hex": sk,
                    "traderId": tId, 
                    "verifyKey_hex": vk,
                    "marketRootId": mkId, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)

In [151]:
pd.DataFrame(response.json(), index=[0])


,checks,marketBranchId,marketMax,marketMin,marketRootId,traderId
0,False,1,1,0,6,7


In [152]:
# Create a trade
url = baseurl+'createTrade'

content_maketrade = {"signingKey_hex": sk, "traderId": tId, "verifyKey_hex": vk,
  "marketRootId": mkId, "marketBranchId": 1, "price": 0.5678, "quantity":1}

In [153]:
# Post trade
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [154]:
# View json response
pd.DataFrame(response.json(), index=[0])

,checks,marketBranchId,marketRootId,price,quantity,traderId
0,"(True, True)",1,6,0.5678,1,7


In [155]:
# Create a matched trade
url = baseurl+'createTrade'
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [156]:
response.json()

{'checks': '(True, True)',
 'marketBranchId': 1,
 'marketRootId': 6,
 'price': 0.5678,
 'quantity': -1,
 'traderId': '7'}

In [157]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 21.39731192588806 seconds.
matched trade: 1 in 19.679604053497314 seconds.
matched trade: 2 in 18.846357107162476 seconds.
matched trade: 3 in 34.86201214790344 seconds.
matched trade: 4 in 18.313287258148193 seconds.
matched trade: 5 in 17.215659856796265 seconds.
matched trade: 6 in 18.703036308288574 seconds.
matched trade: 7 in 16.763543128967285 seconds.
matched trade: 8 in 18.827512741088867 seconds.
matched trade: 9 in 23.355280876159668 seconds.


In [137]:
url = baseurl+'createMarket'

for i in range(4):
    t = time.time()

    content_makemarket = {"signingKey_hex": sk,
      "traderId": int(tId), "verifyKey_hex": vk,
      "marketRootId": i+1, "marketBranchId": 1, "marketMin": 0, "marketMax":0.7}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create market ' +str(i+1))
    

failed to create market 1
failed to create market 2
failed to create market 3
failed to create market 4


In [138]:
response.json()

{'checks': 'False',
 'marketBranchId': 1,
 'marketMax': 0.7,
 'marketMin': 0,
 'marketRootId': 4,
 'traderId': 6}

In [139]:
url = baseurl+'viewMarketBounds'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)



In [140]:
a = pd.read_json(response.json())
a

,marketRootId,marketBranchId,marketMin,marketMax
0,6,1,0,1.0
1,1,1,0,0.7
2,2,1,0,0.7
3,3,1,0,0.7
4,4,1,0,0.7


In [141]:
url = baseurl+'viewOrderBook'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [142]:
a = pd.read_json(response.json())
a

,marketRootId,marketBranchId,price,quantity,traderId
0,6,1,0.5678,1,1
1,6,1,0.5678,-1,1
10,6,1,0.5678,1,1
100,6,1,0.5678,1,3
101,6,1,0.5678,-1,3
102,6,1,0.5678,1,3
103,6,1,0.5678,-1,3
104,6,1,0.5678,-1,3
105,6,1,0.5678,1,3
106,6,1,0.5678,1,3


In [143]:
# Create many unmatched trades
url = baseurl+'createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = 1
    content_maketrade['price'] = 0.44
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('unmatched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


unmatched trade: 0 in 13.099181175231934 seconds.
unmatched trade: 1 in 13.51852011680603 seconds.
unmatched trade: 2 in 14.368139028549194 seconds.
unmatched trade: 3 in 12.055587768554688 seconds.
unmatched trade: 4 in 11.772062063217163 seconds.
unmatched trade: 5 in 12.494034051895142 seconds.
unmatched trade: 6 in 13.005141019821167 seconds.
unmatched trade: 7 in 11.776081800460815 seconds.
unmatched trade: 8 in 12.138066053390503 seconds.
unmatched trade: 9 in 12.684590101242065 seconds.


In [120]:
url = baseurl+'viewOpenTrades'

content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [121]:
pd.read_json(response.json())

,marketRootId,marketBranchId,price,quantity,traderId
0,6,1,0.44,10,5
1,6,1,0.44,10,6


In [144]:
url = baseurl+'viewMatchedTrades'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [145]:
pd.read_json(response.json())

,marketRootId,marketBranchId,price,quantity,traderId
0,6,1,0.5678,0,1
1,6,1,0.5678,1,2
2,6,1,0.5678,0,3
3,6,1,0.5678,-1,6


In [124]:
tId

'6'